In [1]:
%pip install -q python-terrier
%pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from pyterrier_t5 import MonoT5ReRanker, DuoT5ReRanker
monoT5 = MonoT5ReRanker()
duoT5 = DuoT5ReRanker()

In [4]:
dataset = pt.get_dataset("irds:vaswani")
index_loc = "./index_path"
indexer = pt.IterDictIndexer(index_loc, overwrite=True)
indexref = indexer.index(dataset.get_corpus_iter())

[INFO] [starting] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz
                                                            
                                                                                                          [INFO] [finished] http://ir.dcs.gla.ac.uk/resources/test_collections/npl/npl.tar.gz: [00:00] [2.13MB] [4.70MB/s]
vaswani documents: 100%|██████████| 11429/11429 [00:03<00:00, 3616.33it/s] 


In [5]:
bm25 = pt.BatchRetrieve(indexref, wmodel="BM25") % 100
mono_pipeline = bm25 >> pt.text.get_text(dataset, "text") >> monoT5
duo_pipeline = mono_pipeline % 10 >> duoT5

In [6]:
pt.Experiment(
  [
   bm25,
   mono_pipeline,
   duo_pipeline,
  ],
  dataset.get_topics(),
  dataset.get_qrels(),
  names=["BM25", "BM25 >> monoT5", "BM25 >> monoT5 >> duoT5"],
  eval_metrics=["map", "recip_rank", "P.10", "ndcg_cut.10"]
)

[INFO] [starting] building docstore
docs_iter: 100%|█████████████████████| 11429/11429 [00:00<00:00, 206597.88doc/s]
[INFO] [finished] docs_iter: [00:00] [11429doc] [205442.41doc/s]
[INFO] [finished] building docstore s]
duoT5: 100%|██████████| 93/93 [13:22<00:00,  8.63s/queries]


,name,map,recip_rank,P.10,ndcg_cut.10
0,BM25,0.272523,0.725587,0.352688,0.446609
1,BM25 >> monoT5,0.295083,0.750839,0.412903,0.490018
2,BM25 >> monoT5 >> duoT5,0.170299,0.710573,0.412903,0.479814
